<a href="https://colab.research.google.com/github/aditya161205/codoc/blob/main/CODOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GitPython

import os
import ast
from git import Repo

In [24]:
import os
from git import Repo
import shutil

repo_url = "https://github.com/aditya161205/mock"
local_dir = "dir"

# Remove the directory if it exists and is not empty
if os.path.exists(local_dir) and os.path.isdir(local_dir):
    shutil.rmtree(local_dir)

repo = Repo.clone_from(repo_url, local_dir)

In [25]:
import os
import ast
from git import Repo

def parsing(file_path):
  try:
    with open(file_path, "r") as file:
        content = file.read()
    tree=ast.parse(content)

    functions=[]
    classes=[]

    for node in ast.walk(tree):
      if isinstance(node, ast.FunctionDef):
        functions.append(node.name)
      elif isinstance(node, ast.ClassDef):
        classes.append(node.name)

    return functions, classes

  except SyntaxError as e:
    print(f"SyntaxError parsing {file_path}: {e}")
    return [], [] # Return empty lists for functions and classes in case of error
  except Exception as e:
    print(f"Error parsing {file_path}: {e}")
    return [], [] # Handle other potential errors during parsing

In [26]:
for root,dirs,files in os.walk(local_dir):
  for file in files:
    if file.endswith(".py"):
      file_path=os.path.join(root,file)
      functions,classes=parsing(file_path)
      # You can now process the extracted functions and classes.
      # For example, you could print them:
      print("Functions:", functions)
      print("Classes:", classes)

Functions: ['add_numbers', 'multiply_numbers', 'is_even', 'greet', '__init__', 'add', 'multiply', '__init__', 'introduce', 'is_adult']
Classes: ['Calculator', 'Person']
